avoid any additional training and directly print results 
- determine whether the models (T_CPC or TS_CPC or S_CPC) are already capable of generating meaningful representations after the initial unsupervised/self-supervised training.

In [1]:
import wandb
from linearclassifier_all import test_extract, linearclassifier, Wearable
import itertools
import torch
from torch.utils.data import DataLoader


from T_CPC import T_CPC
from TS_CPC import TS_CPC
from S_CPC import S_CPC

In [2]:
torch.backends.cudnn.deterministic = True

torch.manual_seed(0)
np.random.seed(0)

In [3]:
def no_train(model, type, train_loader, test_loader, device):
# def train(num_epochs, model, type, optimizer, train_loader, test_loader, device, test_every=1):

    # Directly extract representations without training
    train_c_t, train_label, _ = test_extract(model, train_loader, device)
    test_c_t, test_label, _ = test_extract(model, test_loader, device)

    # Evaluate linear classifier without updates
    accuracy = linearclassifier(type, train_c_t, train_label, test_c_t, test_label, log_graph=False)    # AUC-ROC, precision, F1 score, etc are saved in wandb already by the linearclassifier function
    print(f"Initial Accuracy without Training: {accuracy * 100:.2f}%")

    # Log metrics directly
    wandb.log({"initial_accuracy": accuracy})


In [4]:
modelname=['T_CPC', 'TS_CPC', 'S_CPC']
type=['ClassifierNN', 'LogisticRegression']
#

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = [256,1024]  # check that it does not influence 

path = '/home/ria/MMBSintern/bachelor/exercise/data' 

train_data = Wearable(path, True)
test_data = Wearable(path, False)


train_loader = DataLoader(train_data, batch_size=64, shuffle=False, num_workers=0)
test_loader = DataLoader(test_data, batch_size=len(test_data), shuffle=False, num_workers=0) 



# Generate all combinations of modelname and batchsize
combinations = list(itertools.product(modelname, type, batch_size))

# Run each combination with each value of num_epochs

for modelname, type, bs in combinations:
        
    print(f"Running with model: {modelname}, classification:{type}, batch size: {bs}")
        
    if modelname == 'T_CPC':
        model = T_CPC(timestep=3, batch_size=bs, seq_len=200).to(device)

    elif modelname == 'TS_CPC':
        model = TS_CPC(timestep=3, batch_size=bs, seq_len=200).to(device)
    
    elif modelname == 'S_CPC':
        model = S_CPC(timestep=3, batch_size=bs, seq_len=200).to(device)

    else:
        raise ValueError(f"Unknown model name: {modelname}")

        
    config = dict( model = modelname,
                    classification = type,
                    batch_size = bs
                    )
                
            
    wandb.init(project="checking", 
                name=f"{modelname,type,bs}",
                config=config,
                reinit=True)


    stats = no_train(model, config['classification'], train_loader, test_loader, device)
            
    wandb.finish() 
            

Running with model: T_CPC, classification:ClassifierNN, batch size: 256


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: riaxchoi (riaxchoi-heinrich-heine-university-d-sseldorf). Use `wandb login --relogin` to force relogin


/home/ria/miniconda3/envs/AILS/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Attempt to open cnn_infer failed: handle=0 error: libcudnn_cnn_infer.so.8: cannot open shared object file: No such file or directory (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:81.)
  return F.conv1d(input, weight, bias, self.stride,


Initial Accuracy without Training: 60.17%


ROC-AUC Score,▁
accuracy,▁
f1_AFib,▁
f1_non-AFib,▁
initial_accuracy,▁
precision_AFib,▁
precision_non-AFib,▁
recall_AFib,▁
recall_non-AFib,▁
ROC-AUC Score,0.66217
accuracy,0.60166


...

ROC-AUC Score,▁
accuracy,▁
f1_AFib,▁
f1_non-AFib,▁
initial_accuracy,▁
precision_AFib,▁
precision_non-AFib,▁
recall_AFib,▁
recall_non-AFib,▁
ROC-AUC Score,0.70538
accuracy,0.65208


Running with model: S_CPC, classification:LogisticRegression, batch size: 1024


/home/ria/miniconda3/envs/AILS/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Initial Accuracy without Training: 58.78%


ROC-AUC Score,▁
accuracy,▁
f1_AFib,▁
f1_non-AFib,▁
initial_accuracy,▁
precision_AFib,▁
precision_non-AFib,▁
recall_AFib,▁
recall_non-AFib,▁
ROC-AUC Score,0.67593
accuracy,0.58781
